In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
import plotly.express as px

from itertools import chain
from unidecode import unidecode

In [2]:
proc_recursos_caminho = '../data/proc/universidade_federal_do_rio_grande_do_norte_-_ufrn'
raw_recursos_caminho = '../data/raw/universidade_federal_do_rio_grande_do_norte_-_ufrn'

In [3]:
resource_re_pattern = lambda pattern: [os.path.join(raw_recursos_caminho,filename) for filename in os.listdir(raw_recursos_caminho) if len(re.findall(pattern,filename))]

In [4]:
read_csv_pattern = lambda pattern : pd.concat(pd.read_csv(filepath,sep=';') for filepath in resource_re_pattern(pattern) if filepath.endswith('.csv'))

In [5]:
estruturas_curriculares = read_csv_pattern('estruturas-curriculares')
componentes_curriculares = read_csv_pattern('componentes-curriculares')
curriculo_componente = read_csv_pattern('curriculo-componente')

In [6]:
estruturas_curriculares = estruturas_curriculares[~estruturas_curriculares['nome_matriz'].isna()]
curriculo_componente = curriculo_componente[curriculo_componente['id_curriculo'].isin(estruturas_curriculares['id_curriculo'])]
componentes_curriculares = componentes_curriculares[componentes_curriculares['id_componente'].isin(curriculo_componente['id_componente_curricular'])]

In [8]:
componentes_curriculares.at[26941,'pre_requisito'] = '( ECT1401 ) E ( ECT1402 )'

In [9]:
def _parse(x):
    x = '' if type(x) != str else x

    import re
    x = x.strip()
    x = re.sub("([A-Z]+[0-9]+\.?[0-9]*)",r"'\1'",x)
    x = re.sub("(\sE\s)",r' and ',x)
    x = re.sub("(\sOU\s)",r' or ',x)

    import ast
    z = ast.parse(x)
    return ast.unparse(z).replace('"""',"'").strip()
    # return x

for col in ['equivalencia', 'pre_requisito', 'co_requisito']:
    print(col)
    componentes_curriculares[col+'_parse'] = componentes_curriculares[col].apply(_parse)
    componentes_curriculares[col+'_len'] = componentes_curriculares[col].apply(lambda e: len(e) if type(e)==str else 0)

equivalencia
pre_requisito
co_requisito


In [10]:
import ast

In [11]:
eq = componentes_curriculares[componentes_curriculares['equivalencia_parse'].apply(len)>100]['equivalencia_parse'].values[0]
eq

"'FIS0001' and 'FIS0002' or 'MEC0301' or 'CIV0302' or ('MEC0351' and 'MEC0352' and 'MEC0359') or ('ECT1402' or 'ECT2412')"

In [12]:
module = ast.parse(eq)
module

In [13]:
module.body

In [14]:
expr = module.body[0]
expr

In [15]:
value = expr.value
value

In [16]:
op = value.op

In [17]:
op

In [18]:
value.values

In [171]:
value.values[1].value

'MEC0301'

In [125]:
componentes_curriculares[componentes_curriculares['equivalencia_parse'].apply(len)>100]['equivalencia'].values

array(['( ( FIS0001 E FIS0002 ) ) OU ( MEC0301 ) OU ( CIV0302 ) OU ( MEC0351 E MEC0352 E MEC0359 ) OU ( ECT1402 OU ECT2412 ) ',
       '( DIM0041 OU DIM0004 OU FIS0610 OU DIM0302 OU CEA0090 OU DIM0044 ) OU ( DIM0044 ) OU ( DIM0404 ) OU ( MAT1527 ) ',
       '( MAT0005 ) OU ( MAT0202 ) OU ( MAT0321 ) OU ( MAT0229 ) OU ( MAT0059 E MAT0060 ) OU ( MAT0346 ) OU ( MAT0341 ) OU ( ECT1202 ) OU ( ECT1212 ) ',
       '( MAT0007 ) OU ( MAT0343 ) OU ( MAT0203 ) OU ( MAT0056 ) OU ( MAT0064 E MAT0065 ) OU ( MAT0230 ) OU ( MAT0319 ) OU ( PRO1003 ) OU ( MAT0221 ) ',
       '( EST0314 ) OU ( EST0313 ) OU ( EST0317 ) OU ( EST0312 ) OU ( PRO1005 ) OU ( ECT1301 ) OU ( EST0311 ) OU ( EST0062 E EST0115 E EST0116 ) OU ( EST0062 E EST0067 ) ',
       '( MAT0061 OU MAT0049 ) OU ( DCA0429 ) OU ( MAT0342 ) OU ( MAT0347 ) OU ( ELE0581 ) OU ( PRO1002 OU MAT0024 ) OU ( MAT0232 ) ',
       '( MAT1513 E MAT1518 ) OU ( MAT0059 ) OU ( MAT0321 E ( MAT0322 OU MAT0060 ) ) OU ( MAT0312 ) OU ( MAT0005 ) OU ( ECT1202 ) ',
  